Pressure compensation to get the water depths during the flood experiments

In [ ]:
import pandas as pd

# will only need data from P3-C 
diver = pd.read_csv('P3C_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')
barom = pd.read_csv('BAROM_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')
barom_interpolated = barom.resample('10s').interpolate(method='time') # to have same sampling rate as diver 
barom_interpolated.rename(columns={'Pressure': 'Barom'}, inplace=True) # rename the pressure column name to barom

In [3]:
# drop temperature columns 
barom_interpolated = barom_interpolated.drop(columns=['Temperature'])
diver = diver.drop(columns=['Temperature'])
# merge barom onto diver data on the same timestamp
pressures = diver.merge(barom_interpolated, left_index=True, right_index=True, how='left')

In [ ]:
# calculate water column (over the diver) depth (in meters)
pressures['WC'] = ((pressures['Pressure'] - pressures['Barom'])*98.0665) / (1000*9.80665)
# calculate total water depth (in meters) 
pressures['Water_Depth'] = pressures['WC'] + 0.5 - 0.293

In [7]:
# re-sample to 1 minute intervals for simplicity
pressures_resampled = pressures.resample('1min').mean()

In [8]:
# export to csv
pressures_resampled.to_csv('P3C_EXP23_depths.csv')